In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#Sahil GDrive
import numpy as np

train_1 = np.load(r'/content/drive/My Drive/Baby Cry/stratified k fold data/train_5.npy')
test_1 = np.load(r'/content/drive/My Drive/Baby Cry/stratified k fold data/test_5.npy')

train_lab_1 = np.load(r'/content/drive/My Drive/Baby Cry/stratified k fold data/train_lab_5.npy')
test_lab_1 = np.load(r'/content/drive/My Drive/Baby Cry/stratified k fold data/test_lab_5.npy')

In [ ]:
import numpy as np

train_1 = np.load(r'/content/drive/My Drive/stratified k fold data/train_5.npy')
test_1 = np.load(r'/content/drive/My Drive/stratified k fold data/test_5.npy')

train_lab_1 = np.load(r'/content/drive/My Drive/stratified k fold data/train_lab_5.npy')
test_lab_1 = np.load(r'/content/drive/My Drive/stratified k fold data/test_lab_5.npy')

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Conv2D, MaxPooling2D, UpSampling2D,Flatten,Reshape,Cropping2D,Conv2DTranspose,concatenate,BatchNormalization,Dropout
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K
from tensorflow.keras.initializers import orthogonal
from tensorflow.keras.optimizers import Adam, SGD

input_file = Input(shape=(288,432, 3))

def Contract(x, filters, kernel , strides, padding, block_id, kernel_init=orthogonal()):
  prefix = f'block_{block_id}_'
  x = tf.keras.layers.Conv2D(filters= filters,kernel_size = kernel, strides = strides, kernel_initializer=kernel_init,
                             activation = None , padding = padding, name= prefix+'conv1_contract')(x)                           
  x = tf.keras.layers.Conv2D(filters= filters,kernel_size = kernel, strides = strides, kernel_initializer=kernel_init,
                             activation = None, padding = padding,  name= prefix+'conv2_contract')(x)
  x = tf.keras.layers.BatchNormalization(name=prefix+'conv_bn_contract')(x)
  x = tf.keras.layers.Dropout(0.2 ,name = prefix +'dropout_contract')(x)
  x = tf.keras.layers.LeakyReLU(name=prefix+'lrelu_contract')(x)
  return x

def Expand(x , y , filters, kernel, trans_kernel,strides, padding, block_id, kernel_init=orthogonal()):
  prefix = f'block_{block_id}_'
  x = tf.keras.layers.Conv2DTranspose(filters,trans_kernel,strides = strides,kernel_initializer = kernel_init, 
                                      padding = padding,name = prefix + 'trans_conv2d_expand')(x)
  x = tf.keras.layers.concatenate([x,y],name = prefix + 'concatenate_expand')
  x = tf.keras.layers.Conv2D(filters,kernel_size = kernel,activation = None ,kernel_initializer = kernel_init,
                             padding = padding,name = prefix + 'conv2d_1_expand')(x)
  x = tf.keras.layers.Conv2D(filters,kernel_size=kernel,activation = None ,kernel_initializer = kernel_init,
                             padding = padding,name = prefix + 'conv2d_2_expand')(x)
  x = tf.keras.layers.BatchNormalization(name=prefix+'conv_bn_expand')(x)
  x = tf.keras.layers.LeakyReLU(name=prefix+'lrelu_expand')(x)
  x = tf.keras.layers.Dropout(0.2, name=prefix+'dropout_expand')((x))
  return x


def U_net_arch(input_shape):
  inputs = Input(shape=input_shape)
  
  #contracting path
  
  c1 = Contract(inputs, 4, 3 , strides = 1, padding='same', block_id=1)
  p1 = tf.keras.layers.MaxPooling2D((2,2),name = 'block_1_max_pool')(c1)

  c2 = Contract(p1, 8, 3 , strides = 1, padding='same', block_id=2)
  p2 = tf.keras.layers.MaxPooling2D((2,2),name = 'block_2_max_pool')(c2)

  c3 = Contract(p2, 16, 3 , strides = 1, padding='same', block_id=3)
  p3 = tf.keras.layers.MaxPooling2D((2,2),name = 'block_3_max_pool')(c3)

  c4 = Contract(p3, 32, 3 , strides = 1, padding='same', block_id=4)
  p4 = tf.keras.layers.MaxPooling2D((2,2),name = 'block_4_max_pool')(c4)

  c5 = Contract(p4, 64, 3 , strides = 1, padding='same', block_id=5)

  #Dense layers to reduce the bottleneck features
  
  x = Flatten(name = 'flatten_bottleneck')(c5)

  
  c5 = Reshape((18,27 , 64))(x)
  
  #expanding path
  
  c6 = Expand(c5, c4, 32, 3 , (2,2), strides = (2,2), padding='same', block_id = 6)
  
  c7 = Expand(c6, c3, 16, 3 , (2,2), strides = (2,2), padding='same', block_id = 7)
  
  c8 = Expand(c7, c2, 8, 3 , (2,2), strides = (2,2), padding='same', block_id = 8)

  c9 = Expand(c8, c1, 4, 3 , (2,2), strides = (2,2), padding='same', block_id = 9)
  
  outputs = tf.keras.layers.Conv2D(3,(1,1),activation = 'sigmoid')(c9)
  
  return Model(inputs=inputs, outputs=outputs) 

input_shape = (288,432)
U_net = U_net_arch((*input_shape,3))
opt=Adam(lr=1e-4)
U_net.compile(optimizer=opt, loss='mse') #msse

U_net.summary()






Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 288, 432, 3) 0                                            
__________________________________________________________________________________________________
block_1_conv1_contract (Conv2D) (None, 288, 432, 4)  112         input_2[0][0]                    
__________________________________________________________________________________________________
block_1_conv2_contract (Conv2D) (None, 288, 432, 4)  148         block_1_conv1_contract[0][0]     
__________________________________________________________________________________________________
block_1_conv_bn_contract (Batch (None, 288, 432, 4)  16          block_1_conv2_contract[0][0]     
_______________________________________________________________________________________

In [ ]:
regularizer = tf.keras.regularizers.l2(.0001)

for layer in U_net.layers:
    if layer.trainable == True:
        for attr in ['kernel_regularizer']:
            if hasattr(layer, attr):
                setattr(layer, attr, regularizer)

In [ ]:
U_net.fit(train_1,train_1, batch_size=32,	validation_data=(test_1,test_1),	epochs=50, verbose=2)

Epoch 1/50
19/19 - 7s - loss: 0.1906 - val_loss: 0.1475
Epoch 2/50
19/19 - 5s - loss: 0.1711 - val_loss: 0.1424
Epoch 3/50
19/19 - 5s - loss: 0.1554 - val_loss: 0.1363
Epoch 4/50
19/19 - 5s - loss: 0.1441 - val_loss: 0.1307
Epoch 5/50
19/19 - 5s - loss: 0.1362 - val_loss: 0.1273
Epoch 6/50
19/19 - 5s - loss: 0.1306 - val_loss: 0.1250
Epoch 7/50
19/19 - 5s - loss: 0.1265 - val_loss: 0.1226
Epoch 8/50
19/19 - 5s - loss: 0.1232 - val_loss: 0.1199
Epoch 9/50
19/19 - 5s - loss: 0.1205 - val_loss: 0.1174
Epoch 10/50
19/19 - 5s - loss: 0.1181 - val_loss: 0.1156
Epoch 11/50
19/19 - 5s - loss: 0.1160 - val_loss: 0.1131
Epoch 12/50
19/19 - 5s - loss: 0.1142 - val_loss: 0.1106
Epoch 13/50
19/19 - 5s - loss: 0.1126 - val_loss: 0.1085
Epoch 14/50
19/19 - 5s - loss: 0.1111 - val_loss: 0.1064
Epoch 15/50
19/19 - 5s - loss: 0.1097 - val_loss: 0.1044
Epoch 16/50
19/19 - 5s - loss: 0.1084 - val_loss: 0.1029
Epoch 17/50
19/19 - 5s - loss: 0.1072 - val_loss: 0.1014
Epoch 18/50
19/19 - 5s - loss: 0.1061 - 

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Conv2D, MaxPooling2D, UpSampling2D,Flatten,Reshape,Cropping2D,Conv2DTranspose,concatenate,BatchNormalization,Dropout
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K
from tensorflow.keras.initializers import orthogonal
from tensorflow.keras.optimizers import Adam, SGD, RMSprop

In [ ]:
batch_size = 10
epochs = 200
inChannel = 3
x, y = 288, 432
input_img = Input(shape = (x, y, inChannel))
num_classes = 5

def encoder(input_img):
    #encoder
    #input = 28 x 28 x 1 (wide and thin)
    conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(input_img) #28 x 28 x 32
    conv1 = BatchNormalization()(conv1)
    conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv1)
    conv1 = BatchNormalization()(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1) #14 x 14 x 32
    conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(pool1) #14 x 14 x 64
    conv2 = BatchNormalization()(conv2)
    conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv2)
    conv2 = BatchNormalization()(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2) #7 x 7 x 64
    conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(pool2) #7 x 7 x 128 (small and thick)
    conv3 = BatchNormalization()(conv3)
    conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv3)
    conv3 = BatchNormalization()(conv3)
    conv4 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv3) #7 x 7 x 256 (small and thick)
    conv4 = BatchNormalization()(conv4)
    conv4 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv4)
    conv4 = BatchNormalization()(conv4)
    return conv4

def decoder(conv4):    
    #decoder
    conv5 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv4) #7 x 7 x 128
    conv5 = BatchNormalization()(conv5)
    conv5 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv5)
    conv5 = BatchNormalization()(conv5)
    conv6 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv5) #7 x 7 x 64
    conv6 = BatchNormalization()(conv6)
    conv6 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv6)
    conv6 = BatchNormalization()(conv6)
    up1 = UpSampling2D((2,2))(conv6) #14 x 14 x 64
    conv7 = Conv2D(32, (3, 3), activation='relu', padding='same')(up1) # 14 x 14 x 32
    conv7 = BatchNormalization()(conv7)
    conv7 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv7)
    conv7 = BatchNormalization()(conv7)
    up2 = UpSampling2D((2,2))(conv7) # 28 x 28 x 32
    decoded = Conv2D(3, (3, 3), activation='sigmoid', padding='same')(up2) # 28 x 28 x 1
    return decoded

autoencoder = Model(input_img, decoder(encoder(input_img)))
autoencoder.compile(loss='mean_squared_error', optimizer = RMSprop())
autoencoder.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 288, 432, 3)]     0         
_________________________________________________________________
conv2d (Conv2D)              (None, 288, 432, 32)      896       
_________________________________________________________________
batch_normalization (BatchNo (None, 288, 432, 32)      128       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 288, 432, 32)      9248      
_________________________________________________________________
batch_normalization_1 (Batch (None, 288, 432, 32)      128       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 144, 216, 32)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 144, 216, 64)     

In [ ]:
regularizer = tf.keras.regularizers.l2(.0001)
batch_size=20
for layer in autoencoder.layers:
    if layer.trainable == True:
        for attr in ['kernel_regularizer']:
            if hasattr(layer, attr):
                setattr(layer, attr, regularizer)
autoencoder.fit(train_1,train_1, batch_size=batch_size,	validation_data=(test_1,test_1),	epochs=100, verbose=2, use_multiprocessing=True, workers=2)

Epoch 1/100
61/61 - 25s - loss: 0.0244 - val_loss: 0.0605
Epoch 2/100
61/61 - 21s - loss: 0.0096 - val_loss: 0.0743
Epoch 3/100
61/61 - 21s - loss: 0.0070 - val_loss: 0.1348
Epoch 4/100
61/61 - 21s - loss: 0.0053 - val_loss: 0.0279
Epoch 5/100
61/61 - 21s - loss: 0.0048 - val_loss: 0.0548
Epoch 6/100
61/61 - 22s - loss: 0.0045 - val_loss: 0.0280
Epoch 7/100
61/61 - 22s - loss: 0.0040 - val_loss: 0.0145
Epoch 8/100
61/61 - 22s - loss: 0.0040 - val_loss: 0.0206
Epoch 9/100
61/61 - 22s - loss: 0.0037 - val_loss: 0.0205
Epoch 10/100
61/61 - 22s - loss: 0.0036 - val_loss: 0.0057
Epoch 11/100
61/61 - 22s - loss: 0.0034 - val_loss: 0.0094
Epoch 12/100
61/61 - 22s - loss: 0.0033 - val_loss: 0.0058
Epoch 13/100
61/61 - 22s - loss: 0.0032 - val_loss: 0.0089
Epoch 14/100
61/61 - 22s - loss: 0.0029 - val_loss: 0.0077
Epoch 15/100
61/61 - 22s - loss: 0.0029 - val_loss: 0.0035
Epoch 16/100
61/61 - 22s - loss: 0.0029 - val_loss: 0.0031
Epoch 17/100
61/61 - 22s - loss: 0.0028 - val_loss: 0.0041
Epoch 

In [ ]:
autoencoder.save_weights('autoencoder.h5')

In [ ]:
from tensorflow.keras.layers import GlobalAveragePooling2D, AveragePooling2D
batch_size = 10
epochs = 200
inChannel = 3
x, y = 288, 432
input_img = Input(shape = (x, y, inChannel))
num_classes = 5

def encoder(input_img):
    #encoder
    #input = 28 x 28 x 1 (wide and thin)
    conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(input_img) #28 x 28 x 32
    conv1 = BatchNormalization()(conv1)
    conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv1)
    conv1 = BatchNormalization()(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1) #14 x 14 x 32
    conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(pool1) #14 x 14 x 64
    conv2 = BatchNormalization()(conv2)
    conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv2)
    conv2 = BatchNormalization()(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2) #7 x 7 x 64
    conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(pool2) #7 x 7 x 128 (small and thick)
    conv3 = BatchNormalization()(conv3)
    conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv3)
    conv3 = BatchNormalization()(conv3)
    conv4 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv3) #7 x 7 x 256 (small and thick)
    conv4 = BatchNormalization()(conv4)
    conv4 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv4)
    conv4 = BatchNormalization()(conv4)
    avgpool = GlobalAveragePooling2D()(conv4)
    return avgpool

def fc(enco):
    flat = Flatten()(enco)
    den = Dense(128, activation='relu')(flat)
    den = Dense(64, activation='relu')(den)
    out = Dense(num_classes, activation='softmax')(den)
    return out

encode = encoder(input_img)
full_model = Model(input_img,fc(encode)) 
for l1,l2 in zip(full_model.layers[:19],autoencoder.layers[0:19]):
    l1.set_weights(l2.get_weights())

for layer in full_model.layers[0:19]:
    layer.trainable = False 

full_model.compile(loss=tf.keras.losses.categorical_crossentropy, optimizer=tf.keras.optimizers.SGD(),metrics=['accuracy'])      
full_model.summary()

Model: "functional_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 288, 432, 3)]     0         
_________________________________________________________________
conv2d_23 (Conv2D)           (None, 288, 432, 32)      896       
_________________________________________________________________
batch_normalization_22 (Batc (None, 288, 432, 32)      128       
_________________________________________________________________
conv2d_24 (Conv2D)           (None, 288, 432, 32)      9248      
_________________________________________________________________
batch_normalization_23 (Batc (None, 288, 432, 32)      128       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 144, 216, 32)      0         
_________________________________________________________________
conv2d_25 (Conv2D)           (None, 144, 216, 64)     

In [ ]:
full_model.compile(loss=tf.keras.losses.categorical_crossentropy, optimizer=tf.keras.optimizers.Adam(),metrics=['accuracy'])
full_model.fit(train_1, train_lab_1, batch_size=20,epochs=50,verbose=2,validation_data=(test_1, test_lab_1))

Epoch 1/50
31/31 - 5s - loss: 0.0912 - accuracy: 0.9868 - val_loss: 0.9469 - val_accuracy: 0.7351
Epoch 2/50
31/31 - 5s - loss: 0.0655 - accuracy: 0.9901 - val_loss: 0.9393 - val_accuracy: 0.7417
Epoch 3/50
31/31 - 5s - loss: 0.0717 - accuracy: 0.9901 - val_loss: 0.9171 - val_accuracy: 0.7417
Epoch 4/50
31/31 - 5s - loss: 0.0864 - accuracy: 0.9802 - val_loss: 1.0491 - val_accuracy: 0.7086
Epoch 5/50
31/31 - 5s - loss: 0.0995 - accuracy: 0.9736 - val_loss: 0.9931 - val_accuracy: 0.7219
Epoch 6/50
31/31 - 5s - loss: 0.0684 - accuracy: 0.9884 - val_loss: 0.9961 - val_accuracy: 0.7351
Epoch 7/50
31/31 - 5s - loss: 0.0600 - accuracy: 0.9950 - val_loss: 0.9903 - val_accuracy: 0.7285
Epoch 8/50
31/31 - 5s - loss: 0.0656 - accuracy: 0.9950 - val_loss: 1.0088 - val_accuracy: 0.7351
Epoch 9/50
31/31 - 5s - loss: 0.1227 - accuracy: 0.9670 - val_loss: 1.1365 - val_accuracy: 0.7152
Epoch 10/50
31/31 - 5s - loss: 0.1698 - accuracy: 0.9488 - val_loss: 1.2487 - val_accuracy: 0.6954
Epoch 11/50
31/31 -

In [ ]:
bottleneck_model = Model(inputs = full_model.input, outputs=full_model.get_layer('flatten_1').output)
def get_every_n(a, n=1):
  for i in range(a.shape[0] // n):
    yield a[n*i:n*(i+1)]
bottleneck=np.zeros((606,256))
bottleneck_test=np.zeros((151,256))
for j,sa in enumerate(get_every_n(train_1)):
   bottleneck[j]=bottleneck_model.predict(sa)
for j,sa in enumerate(get_every_n(test_1)):
   bottleneck_test[j]=bottleneck_model.predict(sa)  

In [ ]:
print(bottleneck.shape)
print(bottleneck_test.shape)

(606, 256)
(151, 256)


In [ ]:
bottleneck_model = Model(inputs = U_net.input, outputs=U_net.get_layer('flatten_bottleneck').output)

bottleneck = bottleneck_model.predict(train_1)

print(bottleneck.shape)


(606, 31104)


In [ ]:
bottleneck_test = bottleneck_model.predict(test_1)
print(bottleneck_test.shape)

(151, 31104)


In [ ]:
train_lab_1 = np.load(r'/content/drive/My Drive/stratified k fold data/train_lab_5.npy')
test_lab_1 = np.load(r'/content/drive/My Drive/stratified k fold data/test_lab_5.npy')
train_lab_1 = train_lab_1.argmax(1)
test_lab_1  =test_lab_1.argmax(1)

print(test_lab_1.shape)

(151,)


In [ ]:
train_lab_1 = train_lab_1.reshape(len(train_lab_1),1)
test_lab_1 = test_lab_1.reshape(len(test_lab_1),1)

In [ ]:
print(train_lab_1.shape)
print(train_lab_1[0:10])

(606, 1)
[[3]
 [3]
 [3]
 [3]
 [3]
 [3]
 [3]
 [3]
 [3]
 [3]]


In [ ]:
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
clf = RandomForestClassifier()

clf.fit(bottleneck,train_lab_1)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  after removing the cwd from sys.path.


RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [ ]:
predicted_test = clf.predict(bottleneck_test)
print(predicted_test.shape)
#predicted_test = predicted_test.reshape((len(predicted_test),1))
#print(predicted_test.shape)


import sklearn
acc_test = sklearn.metrics.accuracy_score(predicted_test,test_lab_1)
print(acc_test)

(151,)
0.695364238410596
